In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from collections import OrderedDict

from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure, show
#from bokeh.palettes import Viridis256, Spectral6, GnBu3, OrRd8, Category10_8
import bokeh.palettes as palettes
from bokeh.transform import factor_cmap
from bokeh.models import Legend, Label



In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
df_ = pd.read_csv("./papers/datasets/one_shot_classif.csv")

In [61]:
df_.head(1)

,page_content,source,format,title,author,subject,keywords,creator,producer,creationDate,modDate,trapped,encryption,start_index,text,category,racional,error_indx
0,"Journal of Apicultural Research , 2015\n[http:...","papers/test_samples/Ruano-Iraheta et al., 2015...",PDF 1.6,Stingless bee distribution and richness in El ...,Carlos Enrique Ruano Iraheta,"Journal of Apicultural Research, 2015. doi:10....",NaN,Arbortext Advanced Print Publisher 9.1.531/W U...,Acrobat Distiller 9.0.0 (Windows),D:20150617193331+05'30',D:20150617193752+05'30',NaN,NaN,1,page_content='Journal of Apicultural Research ...,Metadatos,Se menciona la institución que publicó el artí...,NaN


In [5]:
df2 = df_.copy().filter(['source', 'author','category','racional','page_content'])
df2["file_name"]= df2.source.apply(lambda x: Path(x).name)

df2 = df2.groupby(['file_name', 'category'])\
        [['page_content']]\
        .count().rename({'page_content':'counts'},axis=1).reset_index(['file_name', 'category'])
df2.pivot(columns=['category'],index=['file_name']).droplevel(None, axis= 1)

category,Biología de la especie,Clasificación y descripción de la especie,Distribución de la especie,Ecología y demografía de la especie,Estado de conservación de la especie,Importancia de la especie,Metadatos,Tipo de ambiente en donde se desarrolla la especie
file_name,,,,,,,,
"Biesmeijer & Tóth, 1998.pdf",12.0,NaN,NaN,1.0,NaN,NaN,4.0,1.0
"González-Jaramillo et al., 2019.pdf",1.0,1.0,NaN,NaN,NaN,2.0,2.0,NaN
"Macías-Macías et al., 2011.pdf",12.0,1.0,3.0,NaN,NaN,NaN,4.0,NaN
"Ruano-Iraheta et al., 2015.pdf",3.0,1.0,6.0,2.0,1.0,NaN,11.0,7.0


In [37]:
df3 = df2.pivot(columns=['category'],index=['file_name']).droplevel(None, axis= 1)
df3.index = [name[:-10] for name in df3.index] # sin terminación ", aaaa.pdf"

column_names = df3.columns.to_list()

data = {'index': df3.index.to_list(),
    **{col: df3[col].fillna(0).to_list() for col in column_names}  # Dictionary unpacking
}
source = ColumnDataSource(data)

In [68]:
n_catego = len(column_names)
original_palette = palettes.Category20[n_catego]
gray_color = "#F0F0F0" #"#E0E0E0"

category_colors = {
    category: color
    for category, color in zip(column_names, original_palette)
    if category != "Metadatos"
}

category_colors["Metadatos"] = gray_color
custom_palette = tuple(category_colors[category] for category in OrderedDict(sorted(category_colors.items())))

#mapper = factor_cmap('category', palette="Category10_8", factors=column_names) 
source = ColumnDataSource(data)

p = figure(x_range= data.get('index'))
#p.plot_height=400
#p.plot_width=1000

p.legend.orientation = "horizontal"
p.add_layout(Legend(), place='right')  #'below'

p.xaxis.major_label_orientation = -45
p.xaxis.major_label_text_align='center'
p.legend.click_policy='hide' #"mute" 
#p.legend.ncols=2
p.vbar_stack(
    stackers=column_names, 
    x='index', 
    source=source,
    color = custom_palette,
    legend_label = [cat[:39] for cat in column_names]
    )
show(p)

/home/camilo/.cache/pypoetry/virtualenvs/find-info-app-is0taAXE-py3.10/lib/python3.10/site-packages/bokeh/models/plots.py:815: UserWarning: 
You are attempting to set `plot.legend.orientation` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)
